## Sawtooth demo

In [ ]:
import os
import logging
from logging.handlers import RotatingFileHandler

import elfpy

"""Setup logging"""
filename = "../../.logging/sawtooth_demo.log"
max_bytes = 2e6 # 2MB
log_dir = os.path.dirname(filename)
if not os.path.exists(log_dir):
    os.makedirs(log_dir)
handler = RotatingFileHandler(filename, mode="w", maxBytes=max_bytes)
logging.getLogger().setLevel(logging.DEBUG)  # events of this level and above will be tracked
handler.setFormatter(
    logging.Formatter(elfpy.DEFAULT_LOG_FORMATTER, elfpy.DEFAULT_LOG_DATETIME)
)
logging.getLogger().handlers = [handler,]

In [ ]:
from elfpy.agent import Agent

class Shorter(Agent):
    """
    Agent that is trying to optimize on a rising vault APR via shorts
    """

    def __init__(self, wallet_address, budget=10_000):
        """call basic policy init then add custom stuff"""
        self.pt_to_short = 1_000
        super().__init__(wallet_address, budget)

    def action(self, market, pricing_model):
        """
        implement user strategy
        short if you can, only once
        """
        block_position_list = list(self.wallet.token_in_protocol.values())
        has_opened_short = bool(any((x < -1 for x in block_position_list)))
        can_open_short = self.get_max_pt_short(market, pricing_model) >= self.pt_to_short
        vault_apy = market.share_price * 365 / market.init_share_price
        action_list = []
        if can_open_short:
            if vault_apy > market.get_rate(pricing_model):
                action_list.append(self.create_agent_action(
                    action_type="open_short", trade_amount=self.pt_to_short)
                )
            elif vault_apy < market.get_rate(pricing_model):
                if has_opened_short:
                    action_list.append(
                        self.create_agent_action(action_type="close_short", trade_amount=self.pt_to_short)
                    )
        return action_list

In [ ]:
import numpy as np

from elfpy.utils.parse_config import parse_simulation_config
from elfpy.simulators import Simulator

In [ ]:
config = config_dict={
    'title': 'demo simulation config',
    'market': {
        'min_target_liquidity': 1_000_000.0,
        'max_target_liquidity': 10_000_000.0,
        'min_vault_age': 0,
        'max_vault_age': 1,
        'min_vault_apy': 0.001,
        'max_vault_apy': 0.9,
        'base_asset_price': 1
    },
    'amm': {
        'pricing_model_name': 'Hyperdrive',
        'min_fee': 0.1,
        'max_fee': 0.5,
        'min_pool_apy': 0.02,
        'max_pool_apy': 0.9,
        'floor_fee': 0,
        'verbose': False
    },
    'simulator': {
        'num_trading_days': 10,#180,
        'num_blocks_per_day': 100,#7_200,
        'token_duration': 0.2465753424657534,
        'precision': 64,
        'agent_policies': [],
        'shuffle_users': True,
        'init_lp': True,
        'random_seed': 123,
        'logging_level': 'debug'
    }
}

In [ ]:
ramp = np.linspace(start=0.05, stop=0.1, num=config['simulator']['num_trading_days'])
override_dict = {
    "target_liquidity": 10e6,
    "fee_percent": 0.1,
    "init_pool_apy": 0.05,
    "vault_apy": ramp,
    "num_blocks_per_day": 1,  # temporarily set it to 1 block a day for testing
}

In [ ]:
from elfpy.utils import sim_utils

# instantiate config object
config = sim_utils.override_config_variables(parse_simulation_config(config), override_dict)
# instantiate random number generator
rng = np.random.default_rng(config.simulator.random_seed)
# run random number generators to get random simulation arguments
random_sim_vars = sim_utils.override_random_variables(
    sim_utils.get_random_variables(config, rng), override_dict
)
# instantiate the pricing model
pricing_model = sim_utils.get_pricing_model(model_name=config.amm.pricing_model_name)
# instantiate the market
market = sim_utils.get_market(
    pricing_model,
    random_sim_vars.target_pool_apy,
    random_sim_vars.fee_percent,
    config.simulator.token_duration,
    random_sim_vars.init_share_price,
)
# instantiate the init_lp agent
init_agents = {
    0: sim_utils.get_init_lp_agent(
        config,
        market,
        pricing_model,
        random_sim_vars.target_liquidity,
        random_sim_vars.target_pool_apy,
        random_sim_vars.fee_percent,
    )
}
# set up simulator with only the init_lp_agent
simulator = Simulator(
    config=config,
    pricing_model=pricing_model,
    market=market,
    agents=init_agents,
    rng=rng,
    random_simulation_variables=random_sim_vars,
)
# initialize the market using the LP agent
simulator.collect_and_execute_trades()
simulator.agents.update({1: Shorter(wallet_address=1)})
simulator.agents.update({2: Shorter(wallet_address=2)})

In [ ]:
# run the simulation
simulator.run_simulation()

In [ ]:
from elfpy.utils.post_processing import analysis_dict_to_dataframe

trades = analysis_dict_to_dataframe(simulator.analysis_dict)

In [ ]:
import matplotlib.pyplot as plt

logging.getLogger().setLevel(logging.WARNING)  # events of this level and above will be tracked

fig, ax = plt.subplots()
ax = trades.plot(x="day", y="vault_apy", ax=ax)
ax = trades.plot(x="day", y="pool_apy", ax=ax)